In [4]:
pip install transformer_lens

  Using cached transformer_lens-2.15.0-py3-none-any.whl.metadata (12 kB)
  Using cached beartype-0.14.1-py3-none-any.whl.metadata (28 kB)
  Using cached better_abc-0.0.3-py3-none-any.whl.metadata (1.4 kB)
  Using cached fancy_einsum-0.0.3-py3-none-any.whl.metadata (1.2 kB)
  Using cached jaxtyping-0.3.2-py3-none-any.whl.metadata (7.0 kB)
  Using cached transformers-stream-generator-0.0.5.tar.gz (13 kB)
  Preparing metadata (setup.py) ... done
  Using cached wadler_lindig-0.1.5-py3-none-any.whl.metadata (17 kB)
Using cached transformer_lens-2.15.0-py3-none-any.whl (189 kB)
Using cached beartype-0.14.1-py3-none-any.whl (739 kB)
Using cached better_abc-0.0.3-py3-none-any.whl (3.5 kB)
Using cached fancy_einsum-0.0.3-py3-none-any.whl (6.2 kB)
Using cached jaxtyping-0.3.2-py3-none-any.whl (55 kB)
  Created wheel for transformers-stream-generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12426 sha256=40a47e5c3a7ecc3a76161eeda52bc1ba200f8c0d6f50473b4b89b9437b4c56d9
  

In [2]:
pip install tuned_lens

  Using cached datasets-3.5.1-py3-none-any.whl.metadata (19 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manyli

In [7]:
# Import stuff
import torch
import os
import unicodedata
from IPython.display import HTML
from transformer_lens import HookedTransformer
torch.set_grad_enabled(False)
import json
from transformers import AutoTokenizer, GPTNeoForCausalLM, AutoModelForCausalLM

In [ ]:
def make_prompt(prompt_template: str, subject: str) -> str:
    prompt = prompt_template.format(subject)
    return prompt

In [ ]:
def normalize(s):
    """Normalize answer."""
    import string, re
    s = unicodedata.normalize("NFD", s)
    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)
    def white_space_fix(text):
        return " ".join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    s = white_space_fix(remove_articles(remove_punc(lower(s))))
    return s

In [ ]:
def compute_score(prediction, truth):

    pred_tokens = normalize(prediction).split()
    truth_tokens = normalize(truth).split()

    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    common_tokens = set(pred_tokens) & set(truth_tokens)
    if len(common_tokens) == 0:
        return 0
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return rec

In [ ]:
#load counterfact
data_pth = './counterfact_data.json'
with open(data_pth, 'r') as f:
    counterfact_data = json.load(f)

In [ ]:
MODEL_NAME = "meta-llama/Meta-Llama-3-8B"

# Select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and Hugging Face model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
hf_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    low_cpu_mem_usage=True
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
# Load with transformer_lens
print("Loading HookedTransformer...")

model = HookedTransformer.from_pretrained(
    MODEL_NAME,
    hf_model=hf_model,
    device=device,
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    tokenizer=tokenizer
)

Loading HookedTransformer...


In [ ]:
import os
import json

os.makedirs('./model_output', exist_ok=True)

for case in counterfact_data:
    recall_score = []
    outputs = []
    for p in case['prompts']:
        output = model.generate(p, max_new_tokens=10, temperature=0,prepend_bos=True)
        output = output.replace(p,'')
        recall_score.append(compute_score(output, case['target']))
        outputs.append(output)
    flag = False
    for r in recall_score:
        if r != recall_score[0]:
            flag = True
            break
    case.update({'recall':recall_score,'output':outputs})
    with open('./model_output/llama-8b.jsonl', 'a+') as f:
        f.write(json.dumps(case)+'\n')
    if flag:
        with open('./model_output/llama-8b-hall.jsonl', 'a+') as f:
            f.write(json.dumps(case)+'\n')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
import os
import torch
from tqdm import tqdm
import numpy as np
import pickle, json
from transformers import AutoTokenizer, GPTNeoForCausalLM
from transformer_lens import HookedTransformer
from tuned_lens import TunedLens
import random
from datetime import datetime
import os
from get_dist import *
from plot_utils import *
from data_utils import *

In [ ]:
MODEL_PATH="meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
hf_model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, low_cpu_mem_usage=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
torch.set_grad_enabled(False)
# Set device (CUDA if available, otherwise CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tuned_lens_pth = 'meta-llama/Meta-Llama-3-8B'

In [ ]:
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [ ]:
model = HookedTransformer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B",
    hf_model=hf_model,
    device=device,
    n_devices=1,
    move_to_device=True,
    fold_ln=False,
    center_writing_weights=False,
    center_unembed=False,
    tokenizer=tokenizer
    )

In [ ]:
lens = TunedLens.from_model_and_pretrained(model, tuned_lens_pth)
lens = lens.to(device)

params.pt:   0%|          | 0.00/28.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/204 [00:00<?, ?B/s]

In [ ]:
def get_logits_dist(prompt,output,model,target,device,incl_mid = True, tok_w_space = False):
    if target.startswith(' ') and tok_w_space == False:
        first_tok_idx = 2
    elif target.startswith('0' or '1' or '2' or '3' or '4' or '5' or '6' or '7' or '8' or '9'):
        first_tok_idx = 2
    else:
        first_tok_idx = 1
    llama_tokens = model.to_tokens(prompt)
    ans_tokens = model.to_tokens(target)
    llama_logits, llama_cache = model.run_with_cache(llama_tokens, remove_batch_dim=True)
    llama_tokens_str = model.to_str_tokens(prompt)
    answer_toks_wo_head = torch.tensor([ans_tokens[0][first_tok_idx]]).to(device)
    ans_str = model.to_str_tokens(target)[first_tok_idx]
    accumulated_residual, acc_labels = llama_cache.accumulated_resid(layer=-1, incl_mid=incl_mid, return_labels=True,apply_ln=True)
    scaled_residual_stack = llama_cache.apply_ln_to_stack(accumulated_residual, layer = -1, pos_slice=-1)#好像已经norm了就不会变
    unembed_res = model.unembed(model.ln_final(scaled_residual_stack))
    dist = torch.softmax(unembed_res, dim=-1)
    prob_ans = dist[:,:,answer_toks_wo_head[0]]

    return prob_ans, llama_tokens_str, ans_str

In [ ]:
def get_tuned_logits_dist(lens,prompt,model,target,device, tok_w_space = False):
    if target.startswith(' ') and tok_w_space == False:
        first_tok_idx = 2
    elif target.startswith('0' or '1' or '2' or '3' or '4' or '5' or '6' or '7' or '8' or '9'):
        first_tok_idx = 2
    else:
        first_tok_idx = 1
    llama_tokens = model.to_tokens(prompt)
    ans_tokens = model.to_tokens(target)
    logits, cache = model.run_with_cache(llama_tokens)
    llama_tokens_str = model.to_str_tokens(prompt)
    answer_toks_wo_head = torch.tensor([ans_tokens[0][first_tok_idx]]).to(device)
    ans_str = model.to_str_tokens(target)[first_tok_idx]
    predictition_traj_cache = PredictionTrajectory.from_lens_and_cache(
    lens = lens,
    cache = cache,
    model_logits=logits,
    input_ids=llama_tokens,
    )
    prob_ans = torch.exp(torch.tensor(predictition_traj_cache.log_probs[0][:,:,answer_toks_wo_head[0]]))

    return prob_ans, llama_tokens_str, ans_str

In [ ]:
def get_residual_prob(data,model,target='object'):
    max_tok_len = 0
    tuned_ans_list = []
    logit_ans_list = []
    for d in tqdm(data):
        if target == 'object':
            d_target = ' '+d[target]
        elif target == 'output':
            d_target = d[target]
        tuned_prob_ans, llama_tokens_str, ans_str = get_tuned_logits_dist(lens,d['prompt'],model,d_target, device, tok_w_space = True)
        logit_prob_ans, llama_tokens_str, ans_str = get_logits_dist(d['prompt'],d['output'],model,d_target, device,incl_mid = False, tok_w_space = True)

        max_tok_len = max(max_tok_len,len(llama_tokens_str))
        tuned_ans_list.append(tuned_prob_ans.cpu().numpy())
        logit_ans_list.append(logit_prob_ans.cpu().numpy())
    return tuned_ans_list, logit_ans_list, max_tok_len

In [ ]:
def main():
    data_all = []
    data_r = []
    data_z = []
    data_pth_list = ['model_output/llama-8b-hall.jsonl']

    for pth in data_pth_list:
        with open(pth, 'r') as f:
            for line in f:
                line_d = json.loads(line)
                data_all.append(line_d)

    from operator import itemgetter
    from itertools import groupby

    grouped_data = {}
    data_all.sort(key=itemgetter('relation_id'))
    for key, group in groupby(data_all, key=itemgetter('relation_id')):
        grouped_data[str(key)] = list(group)

    file_pth = './results/llama-8b/tuned_logits/'
    os.makedirs(file_pth,exist_ok=True)
    for relation_id, relation_data in grouped_data.items():
        print('processing...  ',relation_id)
        data_r, data_z = get_right_zero_from_relation(relation_data)
        sample_size = min(len(data_r),len(data_z))
        sample_r = random.sample(data_r, sample_size)
        sample_z = random.sample(data_z, sample_size)
        pkl_pth = file_pth+relation_id+'.pkl'
        if os.path.exists(pkl_pth):
            print('loading from pkl...')
            with open(pkl_pth, 'rb') as f:
                tuned_ans_list_r, logit_ans_list_r, max_tok_len_r, tuned_ans_list_z, logit_ans_list_z, max_tok_len_z, tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o, sample_r, sample_z = pickle.load(f)
        else:
            print('processing...')
            tuned_ans_list_r, logit_ans_list_r, max_tok_len_r = get_residual_prob(sample_r,model)
            tuned_ans_list_z, logit_ans_list_z, max_tok_len_z = get_residual_prob(sample_z,model)
            tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o = get_residual_prob(sample_z,model,'output')

            with open(pkl_pth, 'wb') as f:
                pickle.dump([tuned_ans_list_r, logit_ans_list_r, max_tok_len_r, tuned_ans_list_z, logit_ans_list_z, max_tok_len_z, tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o, sample_r, sample_z], f)


In [ ]:
if __name__ == "__main__":
    main()

processing...   P101


100%|██████████| 27/27 [00:00<00:00, 50421.29it/s]


processing...


100%|██████████| 40/40 [00:06<00:00,  6.26it/s]


processing...   P103


...

100%|██████████| 161/161 [00:23<00:00,  7.00it/s]


processing...   P937


100%|██████████| 254/254 [00:00<00:00, 75766.53it/s]


processing...


100%|██████████| 288/288 [00:42<00:00,  6.84it/s]


In [ ]:
import os
import torch
from tqdm import tqdm
import numpy as np
import pickle, json
from tuned_lens import TunedLens

In [ ]:
import sys
from plot_utils import *
from get_dist import *

# **Both Tuned and Logit Lens**

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

file_pth = './results/llama-8b/tuned_logits/'

l_data = []
label = []
for pkl_pth in os.listdir(file_pth):
    if pkl_pth[-4:] == '.pkl':
        with open(file_pth+pkl_pth, 'rb') as f:
            tuned_ans_list_r, logit_ans_list_r, max_tok_len_r, tuned_ans_list_z, logit_ans_list_z, max_tok_len_z, tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o, sample_r, sample_z = pickle.load(f)
        for i in range(len(tuned_ans_list_r)):
            l_data.append(np.concatenate((tuned_ans_list_r[i][:,-1],logit_ans_list_r[i][:,-1])))
            # l_data.append(logit_ans_list_r[i][:,-1])
            label.append(1)
        for i in range(len(tuned_ans_list_z_o)):
            l_data.append(np.concatenate((tuned_ans_list_z_o[i][:,-1],logit_ans_list_z_o[i][:,-1])))
            # l_data.append(logit_ans_list_z_o[i][:,-1])
            label.append(0)
X_train, X_test, y_train, y_test = train_test_split(l_data, label, test_size=0.2, random_state=42)

clf = svm.SVC(kernel='linear')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)

accuracy: 0.85736013261500205


# **Logit Lens**

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

file_pth = './results/llama-8b/tuned_logits/'


l_data = []
label = []
for pkl_pth in os.listdir(file_pth):
    if pkl_pth[-4:] == '.pkl':
        with open(file_pth+pkl_pth, 'rb') as f:
            tuned_ans_list_r, logit_ans_list_r, max_tok_len_r, tuned_ans_list_z, logit_ans_list_z, max_tok_len_z, tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o, sample_r, sample_z = pickle.load(f)
        for i in range(len(tuned_ans_list_r)):
            #l_data.append(np.concatenate((tuned_ans_list_r[i][:,-1],logit_ans_list_r[i][:,-1])))
            l_data.append(logit_ans_list_r[i][:,-1])
            label.append(1)
        for i in range(len(tuned_ans_list_z_o)):
            #l_data.append(np.concatenate((tuned_ans_list_z_o[i][:,-1],logit_ans_list_z_o[i][:,-1])))
            l_data.append(logit_ans_list_z_o[i][:,-1])
            label.append(0)
X_train, X_test, y_train, y_test = train_test_split(l_data, label, test_size=0.2, random_state=42)

clf = svm.SVC(kernel='linear')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)

accuracy: 0.80734713061533431


# **Tuned Lens**

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

file_pth = './results/llama-8b/tuned_logits/'


l_data = []
label = []
for pkl_pth in os.listdir(file_pth):
    if pkl_pth[-4:] == '.pkl':
        with open(file_pth+pkl_pth, 'rb') as f:
            tuned_ans_list_r, logit_ans_list_r, max_tok_len_r, tuned_ans_list_z, logit_ans_list_z, max_tok_len_z, tuned_ans_list_z_o, logit_ans_list_z_o, max_tok_len_z_o, sample_r, sample_z = pickle.load(f)
        for i in range(len(tuned_ans_list_r)):
            #l_data.append(np.concatenate((tuned_ans_list_r[i][:,-1],logit_ans_list_r[i][:,-1])))
            l_data.append(tuned_ans_list_r[i][:,-1])
            label.append(1)
        for i in range(len(tuned_ans_list_z_o)):
            #l_data.append(np.concatenate((tuned_ans_list_z_o[i][:,-1],logit_ans_list_z_o[i][:,-1])))
            l_data.append(tuned_ans_list_z_o[i][:,-1])
            label.append(0)
X_train, X_test, y_train, y_test = train_test_split(l_data, label, test_size=0.2, random_state=42)

clf = svm.SVC(kernel='linear')

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("accuracy:", accuracy)

print("accuracy: 0.81239813061531924")

accuracy: 0.81239813061531924
